# Setting optimal MLP's hiperparameters for bike dataset

In [5]:
# Importing all necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import pandas as pd

from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

pd.set_option('display.max_columns', 50)

In [23]:
# Create a Neural Network class with flexible settings
class FlexibleRegressionNN(nn.Module):
    def __init__(self, input_size: int, hidden_layers_sizes: list, dropout_rate: float) -> None:
        super(FlexibleRegressionNN, self).__init__()
        layers = list()

        # Define first and n-1 hidden layers
        neuron_num = input_size
        for size in hidden_layers_sizes:
            layers.append(nn.Linear(neuron_num, size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            neuron_num = size

        layers.append(nn.Linear(neuron_num, 10))
        self.model = nn.Sequential(*layers)

    # Define forward loop for NN
    def forward(self, x):
        x = nn.Flatten()(x)
        return self.model(x)

In [18]:
# Define function for calculation RMSE
def root_mean_squared_error(y_true, y_pred):
    mse = torch.mean((y_true - y_pred) ** 2)
    rmse = torch.sqrt(mse)
    return rmse.item()

In [24]:
# Creating function for training and evaluating the model
def train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        
        model.eval()
        with torch.no_grad():
            test_loss = 0.0
            for inputs, targets in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item()
            test_loss /= len(test_loader)
        
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')
    
    return test_loss

# Creating function for selection of best hyperparameters 
def objective(trial):
    # Defining hyperparameters
    hidden_layer_sizes = [
        1024,
        1024,
        1024,
        1024
    ]
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    epochs = trial.suggest_int('epochs', 5, 30)
    input_size = 3072

    # Creating a model and defining of optimizer
    model = FlexibleRegressionNN(input_size, hidden_layer_sizes, dropout_rate)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Convert images to tensors
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Load datasets
    train_set = datasets.CIFAR10(root='data', train=True, download=True, transform=transform)
    test_set = datasets.CIFAR10(root='data', train=False, download=True, transform=transform)

    # Create DataLoaders
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

    # Training launch and return the test loss
    return train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, epochs)

In [26]:
# Settings of searching by Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Getting the best hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-03-12 20:19:23,994] A new study created in memory with name: no-name-366270bc-4072-4bda-b544-cd852958aae4
/var/folders/nd/fsmv2lz93k3_cjwz_1lx75j80000gn/T/ipykernel_62341/2147386946.py:37: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
/var/folders/nd/fsmv2lz93k3_cjwz_1lx75j80000gn/T/ipykernel_62341/2147386946.py:38: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/30, Train Loss: 2.9785, Test Loss: 2.2704
Epoch 2/30, Train Loss: 2.2564, Test Loss: 2.2341
Epoch 3/30, Train Loss: 2.2494, Test Loss: 2.2332
Epoch 4/30, Train Loss: 2.2674, Test Loss: 2.2049
Epoch 5/30, Train Loss: 2.2533, Test Loss: 2.1900
Epoch 6/30, Train Loss: 2.3252, Test Loss: 2.2987
Epoch 7/30, Train Loss: 2.2995, Test Loss: 2.2954
Epoch 8/30, Train Loss: 2.2867, Test Loss: 2.2757
Epoch 9/30, Train Loss: 2.2766, Test Loss: 2.2258
Epoch 10/30, Train Loss: 2.3324, Test Loss: 2.2395
Epoch 11/30, Train Loss: 2.2636, Test Loss: 2.2280
Epoch 12/30, Train Loss: 2.3049, Test Loss: 2.2711
Epoch 13/30, Train Loss: 2.2739, Test Loss: 2.2432
Epoch 14/30, Train Loss: 2.2823, Test Loss: 2.2344
Epoch 15/30, Train Loss: 2.2527, Test Loss: 2.1931
Epoch 16/30, Train Loss: 2.3066, Test Loss: 2.2576
Epoch 17/30, Train Loss: 2.2799, Test Loss: 2.2704
Epoch 18/30, Train Loss: 2.4253, Test Loss: 2.2723
Epoch 19/30, Tr

[I 2025-03-12 20:25:21,675] Trial 0 finished with value: 2.290281867980957 and parameters: {'dropout_rate': 0.333365745661491, 'learning_rate': 0.009130887451985831, 'batch_size': 256, 'epochs': 30}. Best is trial 0 with value: 2.290281867980957.


Epoch 30/30, Train Loss: 2.3263, Test Loss: 2.2903
Files already downloaded and verified
Files already downloaded and verified
Epoch 1/20, Train Loss: 1.8253, Test Loss: 1.6744
Epoch 2/20, Train Loss: 1.6976, Test Loss: 1.5942
Epoch 3/20, Train Loss: 1.6421, Test Loss: 1.5381
Epoch 4/20, Train Loss: 1.6076, Test Loss: 1.5291
Epoch 5/20, Train Loss: 1.5797, Test Loss: 1.5479
Epoch 6/20, Train Loss: 1.5475, Test Loss: 1.5069
Epoch 7/20, Train Loss: 1.5336, Test Loss: 1.4707
Epoch 8/20, Train Loss: 1.5117, Test Loss: 1.4961
Epoch 9/20, Train Loss: 1.4953, Test Loss: 1.4436
Epoch 10/20, Train Loss: 1.4664, Test Loss: 1.4220
Epoch 11/20, Train Loss: 1.4578, Test Loss: 1.4407
Epoch 12/20, Train Loss: 1.4391, Test Loss: 1.4516
Epoch 13/20, Train Loss: 1.4234, Test Loss: 1.4344
Epoch 14/20, Train Loss: 1.4144, Test Loss: 1.4273
Epoch 15/20, Train Loss: 1.3895, Test Loss: 1.4044
Epoch 16/20, Train Loss: 1.3825, Test Loss: 1.4262
Epoch 17/20, Train Loss: 1.3670, Test Loss: 1.4200
Epoch 18/20, Tr

[I 2025-03-12 20:30:29,564] Trial 1 finished with value: 1.4095968340016618 and parameters: {'dropout_rate': 0.2909498069603536, 'learning_rate': 0.001579813513647417, 'batch_size': 128, 'epochs': 20}. Best is trial 1 with value: 1.4095968340016618.


...

Epoch 25/25, Train Loss: 1.0001, Test Loss: 1.2241
Files already downloaded and verified
Files already downloaded and verified
Epoch 1/20, Train Loss: 1.9849, Test Loss: 1.7199
Epoch 2/20, Train Loss: 1.7186, Test Loss: 1.5885
Epoch 3/20, Train Loss: 1.6269, Test Loss: 1.5253
Epoch 4/20, Train Loss: 1.5577, Test Loss: 1.4699
Epoch 5/20, Train Loss: 1.5009, Test Loss: 1.4271
Epoch 6/20, Train Loss: 1.4599, Test Loss: 1.3926
Epoch 7/20, Train Loss: 1.4244, Test Loss: 1.3707
Epoch 8/20, Train Loss: 1.3921, Test Loss: 1.3489
Epoch 9/20, Train Loss: 1.3632, Test Loss: 1.3322
Epoch 10/20, Train Loss: 1.3329, Test Loss: 1.3223
Epoch 11/20, Train Loss: 1.3075, Test Loss: 1.3085
Epoch 12/20, Train Loss: 1.2815, Test Loss: 1.2949
Epoch 13/20, Train Loss: 1.2604, Test Loss: 1.2839
Epoch 14/20, Train Loss: 1.2419, Test Loss: 1.2818
Epoch 15/20, Train Loss: 1.2187, Test Loss: 1.2667
Epoch 16/20, Train Loss: 1.1992, Test Loss: 1.2615
Epoch 17/20, Train Loss: 1.1830, Test Loss: 1.2522
Epoch 18/20, Tr

[I 2025-03-13 02:47:11,701] Trial 48 finished with value: 1.237855089616172 and parameters: {'dropout_rate': 0.46650271569011303, 'learning_rate': 5.1528048246759715e-05, 'batch_size': 128, 'epochs': 20}. Best is trial 33 with value: 1.2149787535682655.


Epoch 20/20, Train Loss: 1.1272, Test Loss: 1.2379
Files already downloaded and verified
Files already downloaded and verified
Epoch 1/22, Train Loss: 1.7615, Test Loss: 1.6038
Epoch 2/22, Train Loss: 1.5812, Test Loss: 1.4857
Epoch 3/22, Train Loss: 1.4966, Test Loss: 1.4498
Epoch 4/22, Train Loss: 1.4355, Test Loss: 1.4118
Epoch 5/22, Train Loss: 1.3906, Test Loss: 1.4007
Epoch 6/22, Train Loss: 1.3458, Test Loss: 1.3650
Epoch 7/22, Train Loss: 1.3013, Test Loss: 1.3819
Epoch 8/22, Train Loss: 1.2682, Test Loss: 1.3492
Epoch 9/22, Train Loss: 1.2351, Test Loss: 1.3607
Epoch 10/22, Train Loss: 1.2054, Test Loss: 1.3550
Epoch 11/22, Train Loss: 1.1857, Test Loss: 1.3454
Epoch 12/22, Train Loss: 1.1501, Test Loss: 1.3185
Epoch 13/22, Train Loss: 1.1248, Test Loss: 1.3561
Epoch 14/22, Train Loss: 1.1056, Test Loss: 1.3198
Epoch 15/22, Train Loss: 1.0735, Test Loss: 1.3440
Epoch 16/22, Train Loss: 1.0639, Test Loss: 1.3318
Epoch 17/22, Train Loss: 1.0427, Test Loss: 1.3470
Epoch 18/22, Tr

[I 2025-03-13 02:52:52,760] Trial 49 finished with value: 1.3673747144167936 and parameters: {'dropout_rate': 0.23940480478152393, 'learning_rate': 0.001023270771627014, 'batch_size': 128, 'epochs': 22}. Best is trial 33 with value: 1.2149787535682655.


Epoch 22/22, Train Loss: 0.9433, Test Loss: 1.3674
Best trial:
  Value: 1.2149787535682655
  Params:
    dropout_rate: 0.43999374377714207
    learning_rate: 8.563695992322964e-05
    batch_size: 32
    epochs: 20
